In [ ]:
from brian2 import *
import scipy.io as sio
%matplotlib inline

Define populations

In [ ]:
nCells = 2
defaultclock.dt = 0.1*ms

E_L = -65*mV;
E_L_S = -56*mV;
C = 100*pfarad

# (unless refractory) flag ensures voltage is clamped during refractory period

eqs = '''
dv/dt = ( (E_L-v) - R*g_ad.*(v-E_k) - R*isyn + R*Itonic ) / (C*R) : mV (unless refractory)'
dg_ad/dt = -g_ad/tau_ad : usiemens
'''

# load spiketimes from matlab
mat_vars = sio.loadmat(mat_fname)
i_spks = mat_vars['ind_spiketimes'] # which neuron is firing?
t_spks = mat_vars['t_spiketimes']  # what time is the neuron firing?

# define cell populations
IC = SpikeGeneratorGroup(nCells,i_spks,t_spks,dt=0.1*ms,sorted=True)
Noise = PoissonGroup(nCells,rates='8+6*randn*Hz',dt=0.1*ms)

S1 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=0.8*ms,reset='v=-50*mV; g_ad += g_inc',method='euler')
S1.v=E_L_S; S1.Itonic = 0*nA; S1.g_inc = 0*usiemens; S1.E_L=E_L_S; S1.R=100*Mohm;
S2 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=0.8*ms,reset='v=-50*mV; g_ad += g_inc',method='euler')
S2.v=E_L_S; S2.Itonic = 0*nA; S2.g_inc = 0*usiemens; S2.E_L=E_L_S; S2.R=100*Mohm;

R1 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=1.5*ms,reset='v=-52*mV; g_ad += g_inc',method='euler')
R1.v=E_L; R1.Itonic = 0*nA; R1.g_inc = 0.0035*usiemens; R1.E_L = E_L; R1.R=200*Mohm;
R2 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=1.5*ms,reset='v=-52*mV; g_ad += g_inc',method='euler')
R2.v=E_L; R2.Itonic = 0*nA; R2.g_inc = 0.0035*usiemens; R2.E_L = E_L; R2.R=200*Mohm;

X1 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=1.5*ms,reset='v=-52*mV; g_ad += g_inc',method='euler')
X1.v=-65*mV; X1.Itonic = 0*nA; X1.g_inc = 0.002*usiemens; X1.E_L = E_L; X1.R=275*Mohm;
X2 = NeuronGroup(nCells,eqs,threshold='v>-47*mV',refractory=1.5*ms,reset='v=-52*mV; g_ad += g_inc',method='euler')
X2.v=-65*mV; X2.Itonic = 0*nA; X2.g_inc = 0.002*usiemens; X2.E_L = E_L; X2.R=275*Mohm;

C = NeuronGroup(1,eqs,threshold='v>-47*mV',refractory=1.5*ms,reset='v=-52*mV; g_ad += g_inc',method='euler')
C.v=-65*mV; C.Itonic = 0*nA; C.g_inc = 0.0035*usiemens; C.E_L = E_L; C.R=200*Mohm;


Define synapses

In [ ]:
E_exc = 0*mV 
E_inh = -80*mV                

# Define equations for double exponentials
syns = '''
scale = (tauD/tauR)**(tauR/(tauD-tauR)) : 1

ds/dt = ( scale * x - s )/tauR : 1
dx/dt = -x/tauD : 1
dF/dt = (1 - F)/tauF : 1
dP/dt = (1 - P)/tauP : 1

isyn =  F * P * gSYN * s * (v - ESYN) : nA
'''

STP = '''
x += 1
F += fF*(5-F)
P *= 1-fP
'''

# initial values for state variables
init_vals = {'s': 0,'x': 0,'F': 1,'P': 1}
syn_params = ['gSYN','ESYN','tauR','tauD','fP','tauP','fF','tauF']

# L4
IC_R1 = Synapses(IC,R1,syns,on_pre=STP)
IC_R1.set_states(init_vals)
setattr(IC_R1,syn_params,[0.011*usiemens,E_exc,0.5*ms,3*ms,0,1*ms,0,1*ms])
IC_R1.connect(j=1) # within-channel connectivity

IC_S1 = Synapses(IC,S1,syns,on_pre=STP)
IC_S1.set_states(init_vals)
setattr(IC_S1,syn_params,[0.02*usiemens,E_exc,0.1*ms,1*ms,0.2,100*ms,0,1*ms])
IC_S1.connect(j=1) # within-channel connectivity

S1_R1 = Synapses(S1,R1,syns,on_pre=STP)
S1_R1.set_states(init_vals)
setattr(S1_R1,syn_params,[0.02*usiemens,E_inh,1*ms,4*ms,0.5,140*ms,0,1*ms])
S1_R1.connect(j=1) # within-channel connectivity

R1_S1 = Synapses(R1,S1,syns,on_pre=STP)
R1_S1.set_states(init_vals)
setattr(R1_S1,syn_params,[0.03*usiemens,E_exc,0.1*ms,1*ms,0.2,100*ms,0,1*ms])
R1_S1.connect(j=1) # within-channel connectivity

R1_X1 = Synapses(R1,X1,syns,on_pre=STP)
R1_X1.set_states(init_vals)
setattr(R1_X1,syn_params,[0.0025*usiemens,E_exc,1.5*ms,4*ms,0,100*ms,0.1,180*ms])
R1_X1.connect(j=1) # within-channel connectivity

X1_R1 = Synapses(X1,R1,syns,on_pre=STP)
X1_R1.set_states(init_vals)
setattr(X1_R1,syn_params,[0.003*usiemens,E_inh,10*ms,50*ms,0,100*ms,0.1,180*ms])
X1_R1.connect(i=2,j=1) # cross-channel connectivity

# L2/3
R1_R2 = Synapses(R1,R2,syns,on_pre=STP)
R1_R2.fF = 0; R1_R2.fP = 0; 
R1_R2.tauF = 180*ms; R1_R2.fP = 80*ms; 
R1_R2.s=0; R1_R2.x=0; R1_R2.F=1; R1_R2.P=1;
IC_R1.gSYN=0.011*usiemens; IC_R1.ESYN=0*mV; IC_R1.tauR=0.5*ms;IC_R1.tauD=3*ms;
R1_R2.connect(j=1) # within-channel connectivity

R1_S2 = Synapses(R1,S2,syns,on_pre=STP)
R1_S2.fF = 0; R1_S2.fP = 0.2; 
R1_S2.tauF = 180*ms; R1_S2.fP = 100*ms; 
R1_S2.s=0; R1_S2.x=0; R1_S2.F=1; R1_S2.P=1;
R1_S2.connect(j=1) # within-channel connectivity

S2_R2 = Synapses(S2,R2,syns,on_pre=STP)
S2_R2.fF = 0; S2_R2.fP = 0.6; 
S2_R2.tauF = 180*ms; S2_R2.fP = 140*ms; 
S2_R2.s=0; S2_R2.x=0; S2_R2.F=1; S2_R2.P=1;
S2_R2.connect(j=1) # within-channel connectivity

R2_S2 = Synapses(R2,S2,syns,on_pre=STP)
R2_S2.fF = 0; R2_S2.fP = 0.2; 
R2_S2.tauF = 180*ms; R2_S2.fP = 100*ms; 
R2_S2.s=0; R2_S2.x=0; R2_S2.F=1; R2_S2.P=1;
R2_S2.connect(j=1) # within-channel connectivity

Noise_R2 = Synapses(Noise,R2,syns,on_pre=STP)
Noise_R2.fF = 0; Noise_R2.fP = 0; 
Noise_R2.tauF = 180*ms; Noise_R2.fP = 100*ms; 
Noise_R2.s=0; Noise_R2.x=0; Noise_R2.F=1; Noise_R2.P=1;
Noise_R2.connect(j=1) # within-channel connectivity

R2_X2 = Synapses(R2,X2,syns,on_pre=STP)
R2_X2.fF = 0.1; R2_X2.fP = 0; 
R2_X2.tauF = 180*ms; R2_X2.fP = 80*ms; 
R2_X2.s=0; R2_X2.x=0; R2_X2.F=1; R2_X2.P=1;
R2_X2.connect(j=1) # within-channel connectivity

X2_R2 = Synapses(X2,R2,syns,on_pre=STP)
X2_R2.fF = 0.1; X2_R2.fP = 0; 
X2_R2.tauF = 180*ms; X2_R2.fP = 80*ms; 
X2_R2.s=0; X2_R2.x=0; X2_R2.F=1; X2_R2.P=1;
X2_R2.connect(i=2,j=1) # cross-channel connectivity

# Convergence onto readout cell

S2_C = Synapses(S2,C,syns,on_pre=STP)
S2_C.fF = 0; S2_C.fP = 0.6; 
S2_C.tauF = 180*ms; S2_C.fP = 140*ms; 
S2_C.s=0; S2_C.x=0; S2_C.F=1; S2_C.P=1;
S2_C.connect() # convergence

R2_C = Synapses(R2,C,syns,on_pre=STP)
R2_C.fF = 0; R2_C.fP = 0; 
R2_C.tauF = 180*ms; R2_C.fP = 100*ms; 
R2_C.s=0; R2_C.x=0; R2_C.F=1; R2_C.P=1;
R2_C.connect()

Run simulations

In [ ]:
run(3500*24*ms)